Data is from testnet, and therefore not accurate.

Install dependencies with:
```sh
pip install -r requirements.txt
```

For some functionality you need a valid API-key.  
Export as environment variables.
Afterwards run notebook from same shell.


Bash/Sh/Zsh:
```sh
export BINANCE_API_KEY='your_api_key'
export BINANCE_API_SECRET='your_secret_key'

# Check
echo $BINANCE_API_KEY
echo $BINANCE_API_SECRET
```
Powershell:
```powershell
$env:BINANCE_API_KEY = 'your_api_key'
$env:BINANCE_API_SECRET = 'your_secret_key'

# Check
$env:BINANCE_API_KEY
$env:BINANCE_API_KEY
```

In [1]:
import os
import pandas as pd
import numpy as np
from binance.spot import Spot as Client
pd.set_option('display.max_rows', 80)
pd.options.display.float_format = '{:.3f}'.format

In [2]:
# Ommit base_url argument to default to api.binance.com
client = Client(os.getenv('BINANCE_API_KEY'), os.getenv('BINANCE_API_SECRET'), base_url='https://testnet.binance.vision')
pd.to_datetime(client.time()['serverTime'], unit='ms')
# client.time()

Timestamp('2022-05-09 16:38:42.475000')

If your time is off you need to synchronize with timeserver

In [3]:
balance = pd.json_normalize(client.account()['balances'])
# balance['free'].astype(float)


In [4]:
symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'LTCUSDT', 'TRXUSDT', 'XRPUSDT' ]
columns =  ['Open time','Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore' ]

In [5]:
assets = pd.concat(([pd.DataFrame(client.klines(symbol, "1d"), columns=columns) for symbol in symbols]), axis = 1, keys=symbols)
assets

BTCUSDT                                                   \
       Open time            Open             High             Low   
0  1651622400000  38127.16000000   67952.01000000   9000.00000000   
1  1651708800000  39690.01000000   48256.09000000  12000.00000000   
2  1651795200000  36552.97000000   90000.00000000   9500.00000000   
3  1651881600000  35851.70000000  179685.00000000   2000.00000000   
4  1651968000000  35472.40000000  172120.00000000   6988.00000000   
5  1652054400000  34036.87000000   44600.00000000   8645.12000000   

                                                                     \
            Close         Volume     Close time  Quote asset volume   
0  39695.80000000  3289.47888900  1651708799999  128626868.06394152   
1  36551.04000000  3747.85911700  1651795199999  142179013.42520536   
2  35844.54000000  5133.87878000  1651881599999  185618082.67083564   
3  35472.39000000  2648.74837000  1651967999999   94693433.70235498   
4  34035.00000000  5928.73934500  1652054399999  204738551.12075512   
5  32137.73000000  5002.60605200  1652140799999  166494023.29861784   

                                                ...     XRPUSDT              \
  Number of trades Taker buy base asset volume  ...        High         Low   
0           109403               2070.21409400  ...  0.64990000  0.60830000   
1           136426               2305.80703300  ...  0.65710000  0.58040000   
2           195320               2791.12783400  ...  0.61630000  0.58130000   
3           110202               1439.33242400  ...  0.60340000  0.56940000   
4           216686               3380.01233600  ...  0.58280000  0.55810000   
5           171914               3020.53393500  ...  0.57870000  0.52030000   

                                                                    \
        Close             Volume     Close time Quote asset volume   
0  0.64610000   7188003.90000000  1651708799999   4497422.00637000   
1  0.59830000  11682556.70000000  1651795199999   7147202.83414000   
2  0.58130000  12824794.10000000  1651881599999   7706389.17155000   
3  0.58180000   4378006.60000000  1651967999999   2579982.36840000   
4  0.56620000   9985261.30000000  1652054399999   5708676.23890000   
5  0.52460000  14080526.20000000  1652140799999   7672380.60101000   

                                                                             \
  Number of trades Taker buy base asset volume Taker buy quote asset volume   
0             8638            7020708.90000000             4393175.81487000   
1            13976           11566933.90000000             7074742.84037000   
2            15203           12610238.80000000             7578080.05335000   
3             5475            4097615.10000000             2413970.94234000   
4            10831            9690239.70000000             5539919.86459000   
5            14900           13884122.00000000             7565125.80908000   

          
  Ignore  
0      0  
1      0  
2      0  
3      0  
4      0  
5      0  

[6 rows x 72 columns]

Formula to convert from unix time to datetime, needs to be set as index

In [6]:
pd.concat([pd.to_datetime(assets[symbol, 'Close time'], unit='ms') for symbol in symbols], axis=1)

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
,Close time,Close time,Close time,Close time,Close time,Close time
0,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999,2022-05-04 23:59:59.999
1,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999,2022-05-05 23:59:59.999
2,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999,2022-05-06 23:59:59.999
3,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999,2022-05-07 23:59:59.999
4,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999,2022-05-08 23:59:59.999
5,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999,2022-05-09 23:59:59.999


Swapping levels for easier selection 

In [7]:
assets = assets.swaplevel(axis=1)
assets

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,...,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,BTCUSDT,...,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT,XRPUSDT
0,1651622400000,38127.16000000,67952.01000000,9000.00000000,39695.80000000,3289.47888900,1651708799999,128626868.06394152,109403,2070.21409400,...,0.64990000,0.60830000,0.64610000,7188003.90000000,1651708799999,4497422.00637000,8638,7020708.90000000,4393175.81487000,0
1,1651708800000,39690.01000000,48256.09000000,12000.00000000,36551.04000000,3747.85911700,1651795199999,142179013.42520536,136426,2305.80703300,...,0.65710000,0.58040000,0.59830000,11682556.70000000,1651795199999,7147202.83414000,13976,11566933.90000000,7074742.84037000,0
2,1651795200000,36552.97000000,90000.00000000,9500.00000000,35844.54000000,5133.87878000,1651881599999,185618082.67083564,195320,2791.12783400,...,0.61630000,0.58130000,0.58130000,12824794.10000000,1651881599999,7706389.17155000,15203,12610238.80000000,7578080.05335000,0
3,1651881600000,35851.70000000,179685.00000000,2000.00000000,35472.39000000,2648.74837000,1651967999999,94693433.70235498,110202,1439.33242400,...,0.60340000,0.56940000,0.58180000,4378006.60000000,1651967999999,2579982.36840000,5475,4097615.10000000,2413970.94234000,0
4,1651968000000,35472.40000000,172120.00000000,6988.00000000,34035.00000000,5928.73934500,1652054399999,204738551.12075512,216686,3380.01233600,...,0.58280000,0.55810000,0.56620000,9985261.30000000,1652054399999,5708676.23890000,10831,9690239.70000000,5539919.86459000,0
5,1652054400000,34036.87000000,44600.00000000,8645.12000000,32137.73000000,5002.60605200,1652140799999,166494023.29861784,171914,3020.53393500,...,0.57870000,0.52030000,0.52460000,14080526.20000000,1652140799999,7672380.60101000,14900,13884122.00000000,7565125.80908000,0


Returns

In [8]:
assets_close = assets["Close"].copy().astype(float)

In [9]:
returns = assets_close.pct_change().dropna()

In [10]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
1,-0.079,-0.066,-0.060,-0.089,-0.136,-0.074
2,-0.019,-0.034,-0.585,-0.013,0.096,-0.028
3,-0.010,-0.007,1.327,-0.012,0.022,0.001
4,-0.041,-0.045,-0.026,-0.005,0.053,-0.027
5,-0.056,-0.061,-0.098,-0.113,-0.120,-0.073


In [11]:
returns.mean(axis = 0)

BTCUSDT   -0.041
ETHUSDT   -0.042
BNBUSDT    0.112
LTCUSDT   -0.046
TRXUSDT   -0.017
XRPUSDT   -0.040
dtype: float64

Annualised Risk & Return

In [12]:
def annualised_risk_return(returns):
    stats = returns.agg(['mean', 'std']).T
    stats.columns = ['Return', 'Risk']
    stats.Return = stats.Return*365.25 # Crypto exchanges trade every day, including leap days.
    stats.loc[stats.Return < -1, 'Return'] = -1 # Set losses > 100% to -100%
    stats.Risk = stats.Risk * np.sqrt(365.25)
    return stats 

In [15]:
stats = annualised_risk_return(returns) # Risk may not be accurate for mean daily losses > 0.274% ()
stats

,Return,Risk
BTCUSDT,-1.000,0.531
ETHUSDT,-1.000,0.453
BNBUSDT,40.788,13.697
LTCUSDT,-1.000,0.967
TRXUSDT,-1.000,2.004
XRPUSDT,-1.000,0.623


Sharpe

In [16]:
riskfree_return = 0.31 # 5 Year Treasury Rate, there is no 5 Year data on testnet, so adjustment is needed.
riskfree_risk = 0
riskfree = [riskfree_return, riskfree_risk]

In [17]:
stats['Sharpe'] = stats['Return'].sub(riskfree[0])/stats['Risk']
stats

,Return,Risk,Sharpe
BTCUSDT,-1.000,0.531,-2.468
ETHUSDT,-1.000,0.453,-2.890
BNBUSDT,40.788,13.697,2.955
LTCUSDT,-1.000,0.967,-1.355
TRXUSDT,-1.000,2.004,-0.654
XRPUSDT,-1.000,0.623,-2.102


Best Sharpe Ratio Portfolio MSRP (All positions held)

In [18]:
assets_close.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT
0,39695.800,2940.670,402.600,106.400,0.086,0.646
1,36551.040,2747.930,378.500,96.900,0.075,0.598
2,35844.540,2655.330,157.070,95.600,0.082,0.581
3,35472.390,2637.330,365.500,94.500,0.083,0.582
4,34035.000,2519.510,356.000,94.000,0.088,0.566


In [19]:
asset_qty = len(assets_close.columns)

In [20]:
portfolio_qty = 100000

In [21]:
matrix = np.random.random(asset_qty * portfolio_qty).reshape (portfolio_qty, asset_qty)

In [22]:
weights = matrix / matrix.sum(axis = 1, keepdims = True)

In [23]:
portfolios_returns = returns.dot(weights.T)

In [24]:
portfolios_stats = annualised_risk_return(portfolios_returns)

In [25]:
portfolios_stats['Sharpe'] = portfolios_stats['Return'].sub(riskfree[0])/portfolios_stats['Risk']

In [26]:
portfolios_stats.describe() 

,Return,Risk,Sharpe
count,100000.000,100000.000,100000.000
mean,-0.199,2.624,-0.458
std,1.996,1.172,0.621
min,-1.000,0.548,-2.392
25%,-1.000,1.644,-0.797
50%,-1.000,2.574,-0.509
75%,-0.951,3.426,-0.359
max,30.738,11.249,2.705


In [27]:
portfolio_max_sharpe_index = portfolios_stats.Sharpe.idxmax()
portfolio_max_sharpe = portfolios_stats.iloc[portfolio_max_sharpe_index]
portfolio_max_sharpe

Return   30.738
Risk     11.249
Sharpe    2.705
Name: 93298, dtype: float64

In [28]:
portfolio_max_sharpe_weights = weights[portfolio_max_sharpe_index, :]

In [29]:
max_sharpe_weights = pd.Series(index = assets_close.columns, data = portfolio_max_sharpe_weights)

In [30]:
returns['MSRP'] = returns.dot(max_sharpe_weights)

In [31]:
returns.head()

,BTCUSDT,ETHUSDT,BNBUSDT,LTCUSDT,TRXUSDT,XRPUSDT,MSRP
1,-0.079,-0.066,-0.060,-0.089,-0.136,-0.074,-0.064
2,-0.019,-0.034,-0.585,-0.013,0.096,-0.028,-0.480
3,-0.010,-0.007,1.327,-0.012,0.022,0.001,1.085
4,-0.041,-0.045,-0.026,-0.005,0.053,-0.027,-0.025
5,-0.056,-0.061,-0.098,-0.113,-0.120,-0.073,-0.095


In [32]:
stats = annualised_risk_return(returns)

In [33]:
stats['Sharpe'] = stats['Return'].sub(riskfree[0])/stats['Risk']
stats

,Return,Risk,Sharpe
BTCUSDT,-1.000,0.531,-2.468
ETHUSDT,-1.000,0.453,-2.890
BNBUSDT,40.788,13.697,2.955
LTCUSDT,-1.000,0.967,-1.355
TRXUSDT,-1.000,2.004,-0.654
XRPUSDT,-1.000,0.623,-2.102
MSRP,30.738,11.249,2.705
